In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_df.shape
#test_df.shape

In [ ]:
x = train_df.drop(['label'],axis=1)
y = train_df['label']

In [ ]:
from keras.datasets import mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
x_train1=np.concatenate([x_train,x_test],axis=0)
y_train1=np.concatenate([y_train,y_test],axis=0)
x_train2 = x_train1.reshape(-1,28*28)
y_train2= y_train1

In [ ]:
y

In [ ]:
x_train3=pd.DataFrame(x_train2,columns=[i for i in x.columns])

In [ ]:
x1 = pd.concat([x,x_train3],axis=0,ignore_index=True)
y1 = pd.concat([y,pd.DataFrame(y_train2)],axis=0,ignore_index=True)

In [ ]:
x1 = x1/255

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(128,activation='elu',input_dim = 784,kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal' ))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(64,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(32,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(16,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10,activation='softmax'))

In [ ]:

model.compile(optimizer='RMSprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'],)

In [ ]:
callback = EarlyStopping(monitor='val_accuracy',min_delta= 0.001,patience=15,verbose=0,mode='auto',baseline=None,restore_best_weights=True)
model.fit(x1,y1,epochs=1000,batch_size=256,callbacks=callback,validation_split=0.2)

In [ ]:
# predict results
test_df = test_df/255
results = model.predict(test_df)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("sample_submission.csv",index=False)